In [1]:
from iSWEEP import *

In [22]:
n = 1e4
s = 0.02
p0 = 0.25
Ne = '../auxillary/threephase-2000G.ne'
Ne = read_Ne(Ne)
short_ibd = 1.0
ploidy=2

In [3]:
# simulate data
simdata = simulate_ibd_from_selective_sweep(n, s, p0, Ne, pairwise_output=True, 
                                            long_ibd = short_ibd,
                                            short_ibd = short_ibd,
                                            ploidy=ploidy
                                           )
print('data simulated')
segments = simdata[3]

# estimate allele frequency
ibdgraph=make_ibd_graph(segments)
print('graph made')
communities=diameter_communities_algorithm(ibdgraph)
print('communities detected')
csizes=[len(comm) for comm in communities]
pest=size_outlier_communities(csizes)/ploidy/n
print('allele frequency estimated')
print('comparison: true vs inferred communities')
print(pest)
sizes=simdata[0][1]
print(size_outlier_communities(sizes)/ploidy/n)

data simulated
graph made
communities detected
allele frequency estimated


In [19]:
# estimate selection coefficient
long_ibd=2.0
long_segments = [seg[2] for seg in segments if seg[2] >= long_ibd]
ab=[long_ibd,np.inf]
obs=len(long_segments)
N=ploidy*n*(ploidy*n-1)/2
p_est=pest
s_est = minimize(chisquared_statistic_for_ibd_from_selective_sweep, 
         (0.01,), 
         args=(p_est,Ne,N,(obs,),ab), 
         bounds=[(0,0.5)], 
         method='Nelder-Mead'
        ).x[0]

In [24]:
# bootstrap
sbs=[]
pbs=[]
B = 5
for b in range(B):
    print(b)
    bootdata = simulate_ibd_from_selective_sweep(n,
                                                 s_est,
                                                 p_est,
                                                 Ne,
                                                 long_ibd=long_ibd,
                                                 short_ibd=short_ibd,
                                                 ploidy=ploidy
                                                )
    bsizes = bootdata[0][1]
    pb = size_outlier_communities(bsizes)/ploidy/n
    sb = minimize(chisquared_statistic_for_ibd_from_selective_sweep, 
         (0.01,), 
         args=(pb,Ne,N,(bootdata[0][0],),ab), 
         bounds=[(0,0.5)], 
         method='Nelder-Mead'
        ).x[0]
    sbs.append(sb)
    pbs.append(pb)
print(sbs)
print(pbs)

0
1
2
3
4
[0.01988720703125001, 0.010515625, 0.01989062500000001, 0.013396484375000002, 0.012306640625]
[0.22525, 0.20475, 0.19905, 0.2806, 0.2671]
